In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display 
import random
import tensorflow as tf

In [13]:
img_list = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
img_list

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
...,...,...,...
22419,p081,c9,img_56936.jpg
22420,p081,c9,img_46218.jpg
22421,p081,c9,img_25946.jpg
22422,p081,c9,img_67850.jpg


In [14]:
img_directory = '../input/state-farm-distracted-driver-detection/imgs/train'
img_directory_test = '../input/state-farm-distracted-driver-detection/imgs/test'

In [15]:
# 4:3 ratio images at 640x480, we'll scale it down to make it easier for the network
train_ds = tf.keras.utils.image_dataset_from_directory(img_directory,validation_split=0.2 ,subset="training",seed=123, image_size =(256,192), batch_size=32)
val_ds = tf.keras.utils.image_dataset_from_directory(img_directory,validation_split=0.2 ,subset="validation",seed=123, image_size =(256,192), batch_size=32)

Found 22424 files belonging to 10 classes.
Using 17940 files for training.
Found 22424 files belonging to 10 classes.
Using 4484 files for validation.


In [16]:
def create_model(input_dims, output_dims):
    with tf.name_scope("cnn"):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=input_dims))
        model.add(tf.keras.layers.Rescaling(1./255))
        model.add(tf.keras.layers.Conv2D(32,3, activation="relu"))
        model.add(tf.keras.layers.MaxPooling2D())
        model.add(tf.keras.layers.Conv2D(64,3, activation="relu"))
        model.add(tf.keras.layers.MaxPooling2D())
        model.add(tf.keras.layers.Conv2D(128,3,activation="relu"))
        model.add(tf.keras.layers.Flatten())
        
        #fully connected layer
        model.add(tf.keras.layers.Dense(128, activation="relu"))
        model.add(tf.keras.layers.Dense(64, activation="relu"))
        model.add(tf.keras.layers.Dense(output_dims, activation="softmax"))
        
        model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
        
    return model

In [17]:
model = create_model([256,192,3], 10)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 256, 192, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 254, 190, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 95, 32)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 93, 64)       18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 46, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 60, 44, 128)       73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 337920)           

In [ ]:
EPOCHS = 25

history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/25
561/561 [==============================] - 64s 113ms/step - loss: 0.4618 - accuracy: 0.8495 - val_loss: 0.0710 - val_accuracy: 0.9799
Epoch 2/25
561/561 [==============================] - 61s 108ms/step - loss: 0.0316 - accuracy: 0.9906 - val_loss: 0.0330 - val_accuracy: 0.9917
Epoch 3/25
561/561 [==============================] - 62s 109ms/step - loss: 0.0245 - accuracy: 0.9933 - val_loss: 0.0692 - val_accuracy: 0.9786
Epoch 4/25
561/561 [==============================] - 59s 104ms/step - loss: 0.0105 - accuracy: 0.9975 - val_loss: 0.0236 - val_accuracy: 0.9944
Epoch 5/25
561/561 [==============================] - 61s 107ms/step - loss: 0.0148 - accuracy: 0.9963 - val_loss: 0.0495 - val_accuracy: 0.9884
Epoch 6/25
561/561 [==============================] - 59s 104ms/step - loss: 0.0240 - accuracy: 0.9928 - val_loss: 0.0439 - val_accuracy: 0.9895
Epoch 7/25
561/561 [==============================] - 60s 105ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.0247 - val_ac

In [ ]:
print(sum(history.history['accuracy']) / EPOCHS)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])

In [ ]:
plt.plot(history.history['loss'])